# County NAICS Files
After running the notebook, delete the python/data_raw folder<br>
Zip level not available -> https://api.census.gov/data/2011/cbp/variables.html<br>
This notebook is a filtered and edited version of us_econ.ipynb for generating yearly data.<br>
For earlier code that generates averaged data aver years please refer us_econ.ipynb.<br>
This notebook will start generating data from year 2017.



In [1]:
import csv
import requests as r
import pandas as pd
import zipfile, io
import os
from tqdm import tqdm
import pathlib
import pandas as pd
import datetime
endyear = datetime.date.today().year
api_headers = {}
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'

In [2]:
# Set a relative location to save the data from the request
repo_dir = pathlib.Path().cwd()
#print(repo_dir)

raw_data_dir = repo_dir / 'data_raw'
out_data_dir = raw_data_dir / 'BEA_Industry_Factors'
    
county_data_dir = out_data_dir / 'county_level'
if not county_data_dir.exists():
    county_data_dir.mkdir(parents=True)

In [3]:
# Load the state FIPS codes key
state_fips = pd.read_csv('../../../us/id_lists/state_fips.csv', usecols=['Name', 'Postal Code', 'FIPS'])
state_fips = state_fips.head(50)  # <-- limit to only US states, not teritories

In [4]:
# Base URL for the API call
base_url = "https://api.census.gov/data"

#
# NOTE Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
#      - stick to 2012 and later for now
#

def get_county_cbp(fips, state, years):
    count = 0
    for year in years:
        print(f"Getting data for state: {state}\tyear: {year}")
        if year >= 2000 and year <= 2002:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS1997_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2003 and year <=2007:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2002_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2008 and year <= 2011:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2007_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2012 and year <= 2016:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2017:
            columns_to_select = "GEO_ID,NAME,COUNTY,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
    
    
        response = r.get(url, headers=api_headers)

        with open(county_data_dir / f"industriesPerCounty_{str.lower(state.replace(' ', ''))}_{year}.csv",'w') as resultPath:
            for line in response.text.strip().split('\n'):
                line=line.replace('[',"").replace(']',"")
                resultPath.write(line + "\n")

        print("  > Finished CSV for year"+str(year))

In [5]:
#Years Initialization for data generation
startyear = 2017
endyear = 2019

In [6]:
for fips in state_fips.FIPS.unique():
     state = state_fips.query(f'FIPS=={fips}').values[0][0]
     years=range(startyear,endyear)
     get_county_cbp(fips, state, years)

Getting data for state: Alabama	year: 2017
  > Finished CSV for year2017
Getting data for state: Alabama	year: 2018
  > Finished CSV for year2018
Getting data for state: Alaska	year: 2017
  > Finished CSV for year2017
Getting data for state: Alaska	year: 2018
  > Finished CSV for year2018
Getting data for state: Arizona	year: 2017
  > Finished CSV for year2017
Getting data for state: Arizona	year: 2018
  > Finished CSV for year2018
Getting data for state: Arkansas	year: 2017
  > Finished CSV for year2017
Getting data for state: Arkansas	year: 2018
  > Finished CSV for year2018
Getting data for state: California	year: 2017
  > Finished CSV for year2017
Getting data for state: California	year: 2018
  > Finished CSV for year2018
Getting data for state: Colorado	year: 2017
  > Finished CSV for year2017
Getting data for state: Colorado	year: 2018
  > Finished CSV for year2018
Getting data for state: Connecticut	year: 2017
  > Finished CSV for year2017
Getting data for state: Connecticut	yea

# Data Aggregation
This part allows us to manage different Fips level (county/state) and different NAICS level (sector/industry/etc...)  
Trick: Getting the NAICS code from all the NAICS files that we downloaded

In [7]:
# Load the data from startyear

def load_all_states(bea_data_dir):
    
    for i in range(startyear,endyear):

        x="_"+str(i)
        files = [f for f in bea_data_dir.iterdir() if x in f.name]

        for f in files:

            # variable selection based on census year
            naics_str = "NAICS2012" if i < 2017 else "NAICS2017"
            naics_ttl = "NAICS2012_TTL" if i < 2017 else "NAICS2017_LABEL"
            geo_ttl = "GEO_TTL" if i < 2017 else "NAME"

            df = pd.read_csv(f,encoding='latin-1',dtype={naics_str: str})
            if 'Unnamed: 11' in df.columns:
                df=df.drop("Unnamed: 11", axis=1)
            if 'Unnamed: 10' in df.columns:
                df=df.drop("Unnamed: 10", axis=1)

            # renaming columns so similar data from 2012 census & 2017 census are entered into appropriate columns
            df = df.rename(columns={"fips": "id", naics_str: "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab", naics_ttl:"NAICS_TTL", geo_ttl:"GEO_TTL"})
            naics_str = "relevant_naics"
            naics_ttl = "NAICS_TTL"
            geo_ttl = "GEO_TTL"

            df['is5'] = df[naics_str].apply(lambda x: 'True' if len(x) == 5 else 'False')

            df.loc[(df['is5'] == 'True') & (df[naics_str].apply(lambda v: v[2:3]) == '-'), 'NAICS_Sector'] = df[naics_str]
            df.loc[(df['is5'] == 'True') & (df[naics_str].apply(lambda v: v[2:3]) != '-'), 'NAICS_Sector'] = df[naics_str].apply(lambda v: v[:2])
            df.loc[(df['is5'] == 'False') , 'NAICS_Sector'] = df[naics_str].apply(lambda v: v[:2])

            yield df
    
df = pd.concat(load_all_states(county_data_dir)).drop("is5", axis=1)

#df

In [8]:
df=df.drop("county", axis=1)

### Process FIPS Code
FIPS is the federal/census unique ID for each geographic area.  States have 2 digives and counties have 5

In [9]:
# Process FIPS code
df['fips'] = df.GEO_ID.apply(lambda GID: GID.split('US')[1])

def county_level(df):
    return df[df['id'].str.len() == 5]

def state_level(df):
    return df[df['id'].str.len() == 2]

In [10]:
# NOTE If this block is run please delete the generated file before pushing into repo (file size too large)
#df.to_csv("allll.csv")

### Renaming Columns for aggregate df
Skipping code block for averaging data for all years from the original us_econ notebook

In [11]:
newDF = df.rename(columns={"fips": "id","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
newDF

,GEO_ID,GEO_TTL,COUNTY,YEAR,relevant_naics,NAICS_TTL,estab,emp,payann,state,NAICS_Sector,id
0,0500000US51127,"New Kent County, Virginia",127,2017,237310,"Highway, street, and bridge construction",4,47,4387,51,23,51127
1,0500000US51127,"New Kent County, Virginia",127,2017,238,Specialty trade contractors,58,295,15102,51,23,51127
2,0500000US51127,"New Kent County, Virginia",127,2017,2381,"Foundation, structure, and building exterior c...",12,79,6415,51,23,51127
3,0500000US51127,"New Kent County, Virginia",127,2017,23814,Masonry contractors,6,24,1010,51,23,51127
4,0500000US51127,"New Kent County, Virginia",127,2017,238140,Masonry contractors,6,24,1010,51,23,51127
...,...,...,...,...,...,...,...,...,...,...,...,...
3880,0500000US44007,"Providence County, Rhode Island",7,2018,81394,Political organizations,10,31,1592,44,81,44007
3881,0500000US44007,"Providence County, Rhode Island",7,2018,813940,Political organizations,10,31,1592,44,81,44007
3882,0500000US44007,"Providence County, Rhode Island",7,2018,81399,"Other similar organizations (except business, ...",5,37,5143,44,81,44007
3883,0500000US44007,"Providence County, Rhode Island",7,2018,813990,"Other similar organizations (except business, ...",5,37,5143,44,81,44007


In [12]:
newDF.tail(50)

,GEO_ID,GEO_TTL,COUNTY,YEAR,relevant_naics,NAICS_TTL,estab,emp,payann,state,NAICS_Sector,id
3835,0500000US44007,"Providence County, Rhode Island",7,2018,812191,Diet and weight reducing centers,4,49,515,44,81,44007
3836,0500000US44007,"Providence County, Rhode Island",7,2018,812199,Other personal care services,57,335,7973,44,81,44007
3837,0500000US44007,"Providence County, Rhode Island",7,2018,8122,Death care services,64,403,18134,44,81,44007
3838,0500000US44007,"Providence County, Rhode Island",7,2018,81221,Funeral homes and funeral services,47,239,10141,44,81,44007
3839,0500000US44007,"Providence County, Rhode Island",7,2018,812210,Funeral homes and funeral services,47,239,10141,44,81,44007
3840,0500000US44007,"Providence County, Rhode Island",7,2018,81222,Cemeteries and crematories,17,164,7993,44,81,44007
3841,0500000US44007,"Providence County, Rhode Island",7,2018,812220,Cemeteries and crematories,17,164,7993,44,81,44007
3842,0500000US44007,"Providence County, Rhode Island",7,2018,8123,Drycleaning and laundry services,103,1211,40556,44,81,44007
3843,0500000US44007,"Providence County, Rhode Island",7,2018,81231,Coin-operated laundries and drycleaners,49,356,6143,44,81,44007
3844,0500000US44007,"Providence County, Rhode Island",7,2018,812310,Coin-operated laundries and drycleaners,49,356,6143,44,81,44007


### Group data by NAICS Sector

NAICS is the North American Industry Classification System. The coarsest level of classification is the *Sector*.

The organization of NAICS is as follows:  <-- from [this page](https://www.census.gov/programs-surveys/economic-census/guidance/understanding-naics.html) on census.gov
- Sector: 2-digit code
    - Subsector: 3-digit code
        - Industry Group: 4-digit code
            - NAICS Industry: 5-digit code
                - National Industry: 6-digit code

Start by grouping the data by sector:

In [13]:
def naics_level(df, naics_level):
    return df[df['relevant_naics'].str.len() == naics_level]

In [14]:
df_naics_2 = naics_level(newDF, 2).reset_index(drop=True)
df_naics_3 = naics_level(newDF, 3).reset_index(drop=True)
df_naics_4 = naics_level(newDF, 4).reset_index(drop=True)
df_naics_5 = naics_level(newDF, 5).reset_index(drop=True)
df_naics_6 = naics_level(newDF, 6).reset_index(drop=True)

df_naics_2 = df_naics_2[df_naics_2.relevant_naics != '00']
df_naics_3 = df_naics_3[df_naics_3.relevant_naics != '00']
df_naics_4 = df_naics_4[df_naics_4.relevant_naics != '00']
df_naics_5 = df_naics_5[df_naics_5.relevant_naics != '00']
df_naics_6 = df_naics_6[df_naics_6.relevant_naics != '00']

In [15]:
#s2=state_level(df_naics_2)
c2=county_level(df_naics_2)
#s3=state_level(df_naics_3)
#c3=county_level(df_naics_3)
#s4=state_level(df_naics_4)
c4=county_level(df_naics_4)
#s5=state_level(df_naics_5)
#c5=county_level(df_naics_5)
s6=state_level(df_naics_6)
#c6=county_level(df_naics_6)

In [16]:
newDF

,GEO_ID,GEO_TTL,COUNTY,YEAR,relevant_naics,NAICS_TTL,estab,emp,payann,state,NAICS_Sector,id
0,0500000US51127,"New Kent County, Virginia",127,2017,237310,"Highway, street, and bridge construction",4,47,4387,51,23,51127
1,0500000US51127,"New Kent County, Virginia",127,2017,238,Specialty trade contractors,58,295,15102,51,23,51127
2,0500000US51127,"New Kent County, Virginia",127,2017,2381,"Foundation, structure, and building exterior c...",12,79,6415,51,23,51127
3,0500000US51127,"New Kent County, Virginia",127,2017,23814,Masonry contractors,6,24,1010,51,23,51127
4,0500000US51127,"New Kent County, Virginia",127,2017,238140,Masonry contractors,6,24,1010,51,23,51127
...,...,...,...,...,...,...,...,...,...,...,...,...
3880,0500000US44007,"Providence County, Rhode Island",7,2018,81394,Political organizations,10,31,1592,44,81,44007
3881,0500000US44007,"Providence County, Rhode Island",7,2018,813940,Political organizations,10,31,1592,44,81,44007
3882,0500000US44007,"Providence County, Rhode Island",7,2018,81399,"Other similar organizations (except business, ...",5,37,5143,44,81,44007
3883,0500000US44007,"Providence County, Rhode Island",7,2018,813990,"Other similar organizations (except business, ...",5,37,5143,44,81,44007


Skipped the code block for `The statewide data does not include NAICS starting with 1!` from us_econ nb

# NAICS code to name translation
Using 2012 naics codes and industries since 2017 naics codes & industries are not available in the current crosswalks data.<br>
TODO: Update the 2012 codes with 2017 codes for updated data

In [17]:
NAICS_codes = pd.read_csv('../../../us/Crosswalk_MasterCrosswalk.csv', usecols=['2012_NAICS_Code', '2012_NAICS_Industry'])

In [18]:
NAICS_codes=NAICS_codes.rename(columns={"2012_NAICS_Code": "relevant_naics", "2012_NAICS_Industry": "industry_detail"})

In [19]:
NAICS_codes=NAICS_codes.dropna()

In [20]:
NAICS_codes=NAICS_codes.drop_duplicates()

In [21]:
NAICS_codes

,relevant_naics,industry_detail
0,11.0,"Agriculture, Forestry, Fishing and Hunting"
13,111.0,Crop Production
19,1111.0,Oilseed and Grain Farming
21,11111.0,Soybean Farming
22,111110.0,Soybean Farming
...,...,...
3885,9281.0,National Security and International Affairs
3887,92811.0,National Security
3888,928110.0,National Security
3889,92812.0,International Affairs


In [22]:
#adding the row for Industries not classified
NAICS_codes
new_row = {'relevant_naics':99, 'industry_detail':"Industries not classified"}

#append row to the dataframe
NAICS_codes = pd.concat([NAICS_codes, pd.DataFrame([new_row])], ignore_index=True)

In [23]:
NAICS_codes

,relevant_naics,industry_detail
0,11.0,"Agriculture, Forestry, Fishing and Hunting"
1,111.0,Crop Production
2,1111.0,Oilseed and Grain Farming
3,11111.0,Soybean Farming
4,111110.0,Soybean Farming
...,...,...
2210,92811.0,National Security
2211,928110.0,National Security
2212,92812.0,International Affairs
2213,928120.0,International Affairs


In [24]:
NAICS_codes.to_csv('../../../us/id_lists/industry_ID_list.csv')

# Making a states json

In [25]:
stateFips = pd.read_csv('../../../us/id_lists/state_fips.csv')

In [26]:
stateFips=stateFips.drop(['Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],axis=1)

In [27]:
stateFips

,Name,Postal Code,FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6
5,Colorado,CO,8
6,Connecticut,CT,9
7,Delaware,DE,10
8,Florida,FL,12
9,Georgia,GA,13


In [28]:
#stateFips.to_json(county_data_dir/'states.json', orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)

# Making county to fips csv

In [29]:
countyDF=c2[['GEO_TTL','id']].drop_duplicates()

In [30]:
countyDF

,GEO_TTL,id
0,"New Kent County, Virginia",51127
15,"Augusta County, Virginia",51015
31,"Northampton County, Virginia",51131
42,"Cumberland County, Virginia",51049
56,"Dickenson County, Virginia",51051
...,...,...
43819,"Wakulla County, Florida",12129
43826,"Escambia County, Florida",12033
45155,"Blaine County, Nebraska",31009
49388,"Harding County, New Mexico",35021


In [31]:
countyDF['hascomma'] = countyDF['GEO_TTL'].apply(lambda x: 'True' if ',' in x else 'False')
countyDF

,GEO_TTL,id,hascomma
0,"New Kent County, Virginia",51127,True
15,"Augusta County, Virginia",51015,True
31,"Northampton County, Virginia",51131,True
42,"Cumberland County, Virginia",51049,True
56,"Dickenson County, Virginia",51051,True
...,...,...,...
43819,"Wakulla County, Florida",12129,True
43826,"Escambia County, Florida",12033,True
45155,"Blaine County, Nebraska",31009,True
49388,"Harding County, New Mexico",35021,True


In [32]:
countyDF.loc[(countyDF['hascomma'] == 'True'), 'county'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[0])
countyDF.loc[(countyDF['hascomma'] == 'True'), 'state'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[-1])

In [33]:
countyDF=countyDF[['state','county','id']].drop_duplicates()

In [34]:
countyDF = countyDF.dropna()

In [35]:
countyDF

,state,county,id
0,Virginia,New Kent County,51127
15,Virginia,Augusta County,51015
31,Virginia,Northampton County,51131
42,Virginia,Cumberland County,51049
56,Virginia,Dickenson County,51051
...,...,...,...
43819,Florida,Wakulla County,12129
43826,Florida,Escambia County,12033
45155,Nebraska,Blaine County,31009
49388,New Mexico,Harding County,35021


In [36]:
stats = stateFips.rename(columns={"Name": "state"})

In [37]:
stats = stats.drop("FIPS",axis=1)
stats

,state,Postal Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,Florida,FL
9,Georgia,GA


In [38]:
countyDF = countyDF.merge(stats, on='state', how='left')

In [39]:
countyDF = countyDF.rename(columns={"Postal Code": "abvr"})
countyDF

,state,county,id,abvr
0,Virginia,New Kent County,51127,VA
1,Virginia,Augusta County,51015,VA
2,Virginia,Northampton County,51131,VA
3,Virginia,Cumberland County,51049,VA
4,Virginia,Dickenson County,51051,VA
...,...,...,...,...
3125,Florida,Wakulla County,12129,FL
3126,Florida,Escambia County,12033,FL
3127,Nebraska,Blaine County,31009,NE
3128,New Mexico,Harding County,35021,NM


# Saving the County level data

In [40]:
states = newDF.state.unique()
#states=[13]

df_naics_6 = df_naics_6.astype({'relevant_naics': 'string'})

a = county_level(df_naics_2)
b = county_level(df_naics_4)
c = county_level(df_naics_6)

for state in states:
    stateName = stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)

    repo_dir = pathlib.Path().cwd()
    state_dir = repo_dir.parents[2] / 'us' / 'state-naics-update' / stateName
    
    if not state_dir.exists():
        state_dir.mkdir(parents=True)
    
    a1 = a[a.state==state]
    b1 = b[b.state==state]
    c1 = c[c.state==state]

    for year in range(startyear, endyear):
        print(year)

        a1y = a1[a1.YEAR==year]
        b1y = b1[b1.YEAR==year]
        c1y = c1[c1.YEAR==year]

        def save_county_data(state, df, counties, naics_level_str):
            for county in counties:
                county = str(county)
                # print(county)
                if len(county) == 2:
                    county = "0" + county
                elif len(county) == 1:
                    county = "00" + county
                
                state = str(state) if len(str(state)) == 2 else "0" + str(state)

                curr_df = df.drop(["GEO_ID", "GEO_TTL", "COUNTY", "YEAR", "NAICS_TTL", "state", "NAICS_Sector"], axis=1)
                curr_df = curr_df.rename(columns={"id":"fips"})

                filename = "US" + state + county + "-" + "census-" + naics_level_str + "-" + str(year) + ".csv"

                curr_df.to_csv(f"../../../us/state-naics-update/{stateName}/{filename}")

        c_a1 = a1.COUNTY.unique()
        c_b1 = b1.COUNTY.unique()
        c_c1 = c1.COUNTY.unique()

        save_county_data(state, a1y, c_a1, "naics2")
        save_county_data(state, b1y, c_b1, "naics4")
        save_county_data(state, c1y, c_c1, "naics6")

VA
2017
2018
KS
2017
2018
AR
2017
2018
RI
2017
2018
DE
2017
2018
CO
2017
2018
SD
2017
2018
NC
2017
2018
ND
2017
2018
MI
2017
2018
ID
2017
2018
TX
2017
2018
IA
2017
2018
MN
2017
2018
PA
2017
2018
CA
2017
2018
AZ
2017
2018
UT
2017
2018
NY
2017
2018
SC
2017
2018
AK
2017
2018
LA
2017
2018
MA
2017
2018
TN
2017
2018
GA
2017
2018
WY
2017
2018
VT
2017
2018
WI
2017
2018
MS
2017
2018
AL
2017
2018
OH
2017
2018
MD
2017
2018
ME
2017
2018
MT
2017
2018
IN
2017
2018
OR
2017
2018
NJ
2017
2018
IL
2017
2018
CT
2017
2018
WA
2017
2018
WV
2017
2018
NV
2017
2018
NE
2017
2018
NH
2017
2018
KY
2017
2018
MO
2017
2018
NM
2017
2018
OK
2017
2018
HI
2017
2018
FL
2017
2018


# saving the statewide data

In [56]:
states=newDF.state.unique()
#states=[13]

for state in states:
    
    b1 = county_level(df_naics_2)
    c1 = b1[b1.state==state]
    c1.astype({'NAICS_Sector': 'int'})
    d1 = c1.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics', "YEAR"],as_index=False).sum()
    d1 = d1.drop(["COUNTY","id", "GEO_TTL", "COUNTY", "NAICS_Sector", "NAICS_TTL", "state", "GEO_ID"],axis=1)
    d1.insert(0, 'fips', state)
    # d1.insert(1, 'COUNTY', 999)
    # d1.insert(2, 'GEO_TTL', 'Statewide')

    b2 = county_level(df_naics_4)
    c2 = b2[b2.state==state]
    c2.astype({'NAICS_Sector': 'int'})
    d2 = c2.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics', "YEAR"],as_index=False).sum()
    d2 = d2.drop(["COUNTY","id", "GEO_TTL", "COUNTY", "NAICS_Sector", "NAICS_TTL", "state", "GEO_ID"],axis=1)
    d2.insert(0, 'fips', state)
    # d2.insert(1, 'COUNTY', 999)
    # d2.insert(2, 'GEO_TTL', 'Statewide')

    b3 = county_level(df_naics_6)
    c3 = b3[b3.state==state]
    c3.astype({'NAICS_Sector': 'int'})
    d3 = c3.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics', "YEAR"],as_index=False).sum()
    d3 = d3.drop(["COUNTY","id", "GEO_TTL", "COUNTY", "NAICS_Sector", "NAICS_TTL", "state", "GEO_ID"],axis=1)
    d3.insert(0, 'fips', state)
    # d3.insert(1, 'COUNTY', 999)
    # d3.insert(2, 'GEO_TTL', 'Statewide')
    
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)

    for year in range(startyear, endyear):
        print(year)

        def save_state_data(state, df, naics_level_str):
                
            state = str(state) if len(str(state)) == 2 else "0" + str(state)

            filename = "US" + state + "-" + "census-" + naics_level_str + "-" + str(year) + ".csv"

            df.to_csv(f"../../../us/state-naics-update/{stateName}/{filename}")

        d1y = d1[d1.YEAR==year]
        d2y = d2[d2.YEAR==year]
        d3y = d3[d3.YEAR==year]

        save_state_data(state, d1y, "naics2")
        save_state_data(state, d2y, "naics4")
        save_state_data(state, d3y, "naics6")

VA
2017
2018
KS
2017
2018
AR
2017
2018
RI
2017
2018
DE
2017
2018
CO
2017
2018
SD
2017
2018
NC
2017
2018
ND
2017
2018
MI
2017
2018
ID
2017
2018
TX
2017
2018
IA
2017
2018
MN
2017
2018
PA
2017
2018
CA
2017
2018
AZ
2017
2018
UT
2017
2018
NY
2017
2018
SC
2017
2018
AK
2017
2018
LA
2017
2018
MA
2017
2018
TN
2017
2018
GA
2017
2018
WY
2017
2018
VT
2017
2018
WI
2017
2018
MS
2017
2018
AL
2017
2018
OH
2017
2018
MD
2017
2018
ME
2017
2018
MT
2017
2018
IN
2017
2018
OR
2017
2018
NJ
2017
2018
IL
2017
2018
CT
2017
2018
WA
2017
2018
WV
2017
2018
NV
2017
2018
NE
2017
2018
NH
2017
2018
KY
2017
2018
MO
2017
2018
NM
2017
2018
OK
2017
2018
HI
2017
2018
FL
2017
2018
